In [1]:
import numpy as np
from numpy import array
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding,Activation, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D 


# 2)-Loading Data


witter Sentiment Data: https://www.kaggle.com/kazanova/sentiment140/data#

In [2]:
df = pd.read_csv('twitter4000.csv')
df.head()

,twitts,sentiment
0,is bored and wants to watch a movie any sugge...,0
1,back in miami. waiting to unboard ship,0
2,"@misskpey awwww dnt dis brng bak memoriessss, ...",0
3,ughhh i am so tired blahhhhhhhhh,0
4,@mandagoforth me bad! It's funny though. Zacha...,0


In [3]:
df.sentiment.value_counts()

1    2000
0    2000
Name: sentiment, dtype: int64

In [4]:
#convert text to list
text = df['twitts'].tolist()

In [5]:
token = Tokenizer()
token.fit_on_texts(text)
y = df['sentiment']



In [6]:
y.shape

(4000,)

In [7]:
token

In [8]:
vocab_size = len(token.word_index) + 1
vocab_size

10135

In [9]:
type(token.index_word)

dict

In [10]:
index_word_list=token.index_word

In [11]:
token_df=pd.DataFrame.from_dict(index_word_list,orient='index', columns=["words"])
token_df.head()

,words
1,i
2,to
3,the
4,a
5,my


In [12]:
token_df.tail()

,words
10130,saravananr
10131,heycameron
10132,thejetset
10133,templating
10134,omarepps


In [13]:
#x = ['i to the a and']
#encoded format: x = [1, 2, 3, 4, 6]
encoded_text = token.texts_to_sequences(text)

In [14]:
max_length = 120
X = pad_sequences(encoded_text, maxlen=max_length, padding='post')

In [15]:
print(X)

[[    8   304     6 ...     0     0     0]
 [   52    10  1019 ...     0     0     0]
 [ 3082  1197   668 ...     0     0     0]
 ...
 [ 1033    21  1021 ...     0     0     0]
 [10134   134     7 ...     0     0     0]
 [   94    11   226 ...     0     0     0]]


In [16]:
type(X)

numpy.ndarray

In [17]:
X.shape

(4000, 120)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [23]:
y_train_arr = np.asarray(y_train)
y_test_arr = np.asarray(y_test)

In [24]:
vec_size = 300

model = Sequential()
model.add(Embedding(vocab_size, vec_size, input_length=max_length))

model.add(Conv1D(64, 8, activation = 'relu'))
model.add(MaxPooling1D(2))
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(16, activation='relu'))

model.add(GlobalMaxPooling1D())

model.add(Dense(1, activation='sigmoid'))

In [25]:
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [26]:
%%time
model.fit(X_train, y_train_arr, epochs = 5, validation_data = (X_test, y_test_arr))

Train on 3200 samples, validate on 800 samples
Epoch 1/5
3200/3200 [==============================] - 10s 3ms/sample - loss: 0.6955 - accuracy: 0.5097 - val_loss: 0.6935 - val_accuracy: 0.4950
Epoch 2/5
3200/3200 [==============================] - 9s 3ms/sample - loss: 0.6942 - accuracy: 0.4991 - val_loss: 0.6860 - val_accuracy: 0.5813
Epoch 3/5
3200/3200 [==============================] - 9s 3ms/sample - loss: 0.6426 - accuracy: 0.6344 - val_loss: 0.5781 - val_accuracy: 0.7063
Epoch 4/5
3200/3200 [==============================] - 9s 3ms/sample - loss: 0.3950 - accuracy: 0.8322 - val_loss: 0.5720 - val_accuracy: 0.6938
Epoch 5/5
3200/3200 [==============================] - 9s 3ms/sample - loss: 0.1692 - accuracy: 0.9406 - val_loss: 0.6482 - val_accuracy: 0.7225
CPU times: user 3min 51s, sys: 36.7 s, total: 4min 28s
Wall time: 47.1 s


# Check out of sample text

In [27]:
def get_encoded(x):
    x = token.texts_to_sequences(x)
    x = pad_sequences(x, maxlen=max_length, padding = 'post')
    return x

In [28]:
x = ['worst services. will not come again']

In [29]:
model.predict_classes(get_encoded(x))

array([[0]], dtype=int32)

In [30]:
x = ['thank you for watching']

In [31]:
model.predict_classes(get_encoded(x))

array([[1]], dtype=int32)